In [0]:
df = spark.read.format('parquet').option('header','true').option('inferSchema', 'true')\
    .load('dbfs:/FileStore/iris.parquet')
df.show()

+------------+-----------+------------+-----------+-------+
|sepal.length|sepal.width|petal.length|petal.width|variety|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| Setosa|
|         4.9|        3.0|         1.4|        0.2| Setosa|
|         4.7|        3.2|         1.3|        0.2| Setosa|
|         4.6|        3.1|         1.5|        0.2| Setosa|
|         5.0|        3.6|         1.4|        0.2| Setosa|
|         5.4|        3.9|         1.7|        0.4| Setosa|
|         4.6|        3.4|         1.4|        0.3| Setosa|
|         5.0|        3.4|         1.5|        0.2| Setosa|
|         4.4|        2.9|         1.4|        0.2| Setosa|
|         4.9|        3.1|         1.5|        0.1| Setosa|
|         5.4|        3.7|         1.5|        0.2| Setosa|
|         4.8|        3.4|         1.6|        0.2| Setosa|
|         4.8|        3.0|         1.4|        0.1| Setosa|
|         4.3|        3.0|         1.1| 

In [0]:
df.columns

Out[3]: ['sepal.length', 'sepal.width', 'petal.length', 'petal.width', 'variety']

In [0]:
df.groupBy('variety').agg({'sepal.length': 'count' }).show()

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4396117031493980>:1
----> 1 df.groupBy('variety').agg({'sepal.length': 'count' }).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/group.py:172, in GroupedData.agg(self, *exprs)
    170 assert exprs, "exprs should not be empty"
    171 if len(exprs) == 1 and isinstance(exprs[0], dict):
--> 172     jdf = self._jgd.agg(exprs[0])
    173 else:
    174     # Columns
    175     assert all(isinstance(c, Column) for c in exprs), "all exprs should be Column"

Fil

In [0]:
df.groupBy('variety').agg({'*': 'count' }).sort('variety').show()

+----------+--------+
|   variety|count(1)|
+----------+--------+
|    Setosa|      50|
|Versicolor|      50|
| Virginica|      50|
+----------+--------+



In [0]:
df.schema

Out[4]: StructType([StructField('sepal.length', DoubleType(), True), StructField('sepal.width', DoubleType(), True), StructField('petal.length', DoubleType(), True), StructField('petal.width', DoubleType(), True), StructField('variety', StringType(), True)])

In [0]:
'sepal.length'.replace('.','_')

Out[5]: 'sepal_length'

In [0]:
rename_dict={}
for i in df.columns:
    print(i, type(i))
    rename_dict[i]= i.replace('.','_')

sepal.length <class 'str'>
sepal.width <class 'str'>
petal.length <class 'str'>
petal.width <class 'str'>
variety <class 'str'>


In [0]:
print(rename_dict)

{'sepal.length': 'sepal_length', 'sepal.width': 'sepal_width', 'petal.length': 'petal_length', 'petal.width': 'petal_width', 'variety': 'variety'}


In [0]:

for old_name, new_name in rename_dict.items():
    print(old_name, new_name)
    df = df.withColumnRenamed(old_name, new_name)

sepal.length sepal_length
sepal.width sepal_width
petal.length petal_length
petal.width petal_width
variety variety


In [0]:
df.columns

Out[52]: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'variety']

In [0]:
type(df.groupBy('variety'))

Out[8]: pyspark.sql.group.GroupedData

In [0]:
from pyspark.sql.functions import col
df.groupBy('variety').max('sepal_length').sort('variety').show()

+----------+-----------------+
|   variety|max(sepal_length)|
+----------+-----------------+
|    Setosa|              5.8|
|Versicolor|              7.0|
| Virginica|              7.9|
+----------+-----------------+



In [0]:
df.groupBy('variety').max('sepal_length').orderBy('variety').show()

+----------+-----------------+
|   variety|max(sepal_length)|
+----------+-----------------+
|    Setosa|              5.8|
|Versicolor|              7.0|
| Virginica|              7.9|
+----------+-----------------+



In [0]:
df.groupBy('variety').max('sepal_length', 'sepal_width').show()

+----------+-----------------+----------------+
|   variety|max(sepal_length)|max(sepal_width)|
+----------+-----------------+----------------+
| Virginica|              7.9|             3.8|
|    Setosa|              5.8|             4.4|
|Versicolor|              7.0|             3.4|
+----------+-----------------+----------------+



In [0]:
### for multiple column
df1 = df.groupBy('col1','col2').count()

In [0]:
df.describe().show()

+-------+------------------+-------------------+------------------+------------------+---------+
|summary|      sepal_length|        sepal_width|      petal_length|       petal_width|  variety|
+-------+------------------+-------------------+------------------+------------------+---------+
|  count|               150|                150|               150|               150|      150|
|   mean| 5.843333333333335|  3.057333333333334|3.7580000000000027| 1.199333333333334|     null|
| stddev|0.8280661279778637|0.43586628493669793|1.7652982332594662|0.7622376689603467|     null|
|    min|               4.3|                2.0|               1.0|               0.1|   Setosa|
|    max|               7.9|                4.4|               6.9|               2.5|Virginica|
+-------+------------------+-------------------+------------------+------------------+---------+



In [0]:
from pyspark.sql.functions import count, min, max
df.groupBy('variety').agg(count('*').alias('tot_records'),\
                    min('sepal_length').alias("MIN_SEPAL_LENGTH"),\
                        max('sepal_length').alias("MAX_SEPAL_LENGTH")).show()

+----------+-----------+----------------+----------------+
|   variety|tot_records|MIN_SEPAL_LENGTH|MAX_SEPAL_LENGTH|
+----------+-----------+----------------+----------------+
| Virginica|         50|             4.9|             7.9|
|    Setosa|         50|             4.3|             5.8|
|Versicolor|         50|             4.9|             7.0|
+----------+-----------+----------------+----------------+



### Pivot and Unpivot

In [0]:
df_bank = spark.read.format('csv').option('header','true').option('inferSchema', 'true')\
    .load('dbfs:/FileStore/bank.csv')
df_bank.show()

+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|   unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|     services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|   management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30|   management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|  blue-collar|married|secondary|     no|      0|    yes|  no| unknown| 

In [0]:
df_bank.columns

Out[3]: ['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y']

In [0]:
### job wise education wise count

df_bank.groupby('job', 'education').count().show()

+------------+---------+-----+
|         job|education|count|
+------------+---------+-----+
|     retired|  primary|   80|
|     student|  unknown|   16|
|  management| tertiary|  787|
|entrepreneur|secondary|   58|
|  technician|secondary|  520|
|     student|secondary|   47|
|     student| tertiary|   19|
|      admin.| tertiary|   51|
|     unknown|  unknown|   15|
|  technician| tertiary|  211|
| blue-collar| tertiary|   12|
|     unknown|secondary|    8|
|  unemployed|  unknown|    2|
|      admin.|secondary|  393|
|  management|  unknown|   27|
|  management|secondary|  116|
|entrepreneur| tertiary|   73|
| blue-collar|secondary|  524|
|entrepreneur|  unknown|   11|
|   housemaid|  primary|   57|
+------------+---------+-----+
only showing top 20 rows



In [0]:
## the same example can be done using  pivot
df_bank.groupby('job').pivot('education').count().show()

+-------------+-------+---------+--------+-------+
|          job|primary|secondary|tertiary|unknown|
+-------------+-------+---------+--------+-------+
|   management|     39|      116|     787|     27|
|      retired|     80|      105|      31|     14|
|      unknown|      7|        8|       8|     15|
|self-employed|     15|       76|      88|      4|
|      student|      2|       47|      19|     16|
|  blue-collar|    369|      524|      12|     41|
| entrepreneur|     26|       58|      73|     11|
|       admin.|     17|      393|      51|     17|
|   technician|     15|      520|     211|     22|
|     services|     25|      363|      16|     13|
|    housemaid|     57|       28|      22|      5|
|   unemployed|     26|       68|      32|      2|
+-------------+-------+---------+--------+-------+



In [0]:
df_job_edu = df_bank.groupby('job').pivot('education',['primary','unknown']).count()
df_job_edu.show()

+-------------+-------+-------+
|          job|primary|unknown|
+-------------+-------+-------+
|   management|     39|     27|
|      retired|     80|     14|
|      unknown|      7|     15|
|self-employed|     15|      4|
|      student|      2|     16|
|  blue-collar|    369|     41|
| entrepreneur|     26|     11|
|       admin.|     17|     17|
|   technician|     15|     22|
|     services|     25|     13|
|    housemaid|     57|      5|
|   unemployed|     26|      2|
+-------------+-------+-------+



### Unpivot

In [0]:
m=10
x=2
c=1
eval("m*x+c")

Out[21]: 21

In [0]:
from pyspark.sql.functions import expr

un_df = df_job_edu.select('job', expr("stack(2,'pri',primary,'not recorder',unknown)as (education,total)"  ))
un_df.show()

+-------------+------------+-----+
|          job|   education|total|
+-------------+------------+-----+
|   management|         pri|   39|
|   management|not recorder|   27|
|      retired|         pri|   80|
|      retired|not recorder|   14|
|      unknown|         pri|    7|
|      unknown|not recorder|   15|
|self-employed|         pri|   15|
|self-employed|not recorder|    4|
|      student|         pri|    2|
|      student|not recorder|   16|
|  blue-collar|         pri|  369|
|  blue-collar|not recorder|   41|
| entrepreneur|         pri|   26|
| entrepreneur|not recorder|   11|
|       admin.|         pri|   17|
|       admin.|not recorder|   17|
|   technician|         pri|   15|
|   technician|not recorder|   22|
|     services|         pri|   25|
|     services|not recorder|   13|
+-------------+------------+-----+
only showing top 20 rows



In [0]:
df = spark.range(start=10, end=101,  step=10)
df.show()

+---+
| id|
+---+
| 10|
| 20|
| 30|
| 40|
| 50|
| 60|
| 70|
| 80|
| 90|
|100|
+---+



In [0]:
df1= df.sample(fraction=0.1, seed=123)
df1.show()

+---+
| id|
+---+
| 50|
| 90|
+---+



+---+
| id|
+---+
| 50|
| 90|
+---+



In [0]:
df.show()

+---+
| id|
+---+
| 10|
| 20|
| 30|
| 40|
| 50|
| 60|
| 70|
| 80|
| 90|
|100|
+---+



In [0]:
df_bank.show()

+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|   unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|     services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|   management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
| 30|   management|married| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no|
| 59|  blue-collar|married|secondary|     no|      0|    yes|  no| unknown| 

In [0]:
from pyspark.sql.functions import upper
def conv_upper(df):
    return df.withColumn('marital',upper(df.marital))

def calc_bal(df):
    return df.withColumn('bal', df.balance*2 )
df_tran=df_bank.transform(conv_upper).transform(calc_bal)
df_tran.show()


+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-----+
|age|          job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|  bal|
+---+-------------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+-----+
| 30|   unemployed|MARRIED|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no| 3574|
| 33|     services|MARRIED|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no| 9578|
| 35|   management| SINGLE| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no| 2700|
| 30|   management|MARRIED| tertiary|     no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown| no| 2952|
| 59|  blue-collar|MARRIED|seconda

In [0]:
from pyspark.sql.functions import transform
### this is used to apply transformation on array type of data

##from pyspark.sql.functions import col, array
data1 = [(2,'Sunita', ['football','Archary']),(3,'Fatima',['Baseball','chess'])]
columns =['id', 'name', 'games']

df = spark.createDataFrame(data = data1, schema = columns)
display(df)
df.printSchema()

id,name,games
2,Sunita,"List(football, Archary)"
3,Fatima,"List(Baseball, chess)"


root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- games: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [0]:
df.select('id','name', transform('games', lambda x:upper(x)).alias("GAMES")).show()

+---+------+-------------------+
| id|  name|              GAMES|
+---+------+-------------------+
|  2|Sunita|[FOOTBALL, ARCHARY]|
|  3|Fatima|  [BASEBALL, CHESS]|
+---+------+-------------------+



Check the difference in tempview and global temp view 

## Empty Data Frame

In [0]:
df_empty = spark.createDataFrame([], 'a STRING')
df_empty.isEmpty()


Out[23]: True

In [0]:
df_empty.show()

+---+
|  a|
+---+
+---+



In [0]:
df = spark.sql("SHOW TABLES")
df.isLocal()

Out[2]: True